In [1]:
from mmpose.datasets.datasets.body import Mo2Cap2CocoDataset
from mmengine import Config
from mmengine import build_from_cfg, is_seq_of
from mmpose.registry import DATASETS
from mmpose.datasets.transforms.bottomup_transforms import (BottomupGetHeatmapMask, BottomupRandomAffine,
                                  BottomupRandomChoiceResize,
                                  BottomupRandomCrop, BottomupResize)
from mmpose.datasets.transforms.common_transforms import (Albumentation, FilterAnnotations,
                                GenerateTarget, GetBBoxCenterScale,
                                PhotometricDistortion, RandomBBoxTransform,
                                RandomFlip, RandomHalfBody, YOLOXHSVRandomAug)

from mmpose.datasets.transforms.custom_transforms import EgoposeFilterAnnotations,FisheyeCropTransform

from mmpose.datasets.transforms.converting import KeypointConverter, SingleHandConverter
from mmpose.datasets.transforms.formatting import PackPoseInputs
from mmpose.datasets.transforms.hand_transforms import HandRandomFlip
from mmpose.datasets.transforms.loading import LoadImage
from mmpose.datasets.transforms.mix_img_transforms import Mosaic, YOLOXMixUp
from mmpose.datasets.transforms.pose3d_transforms import RandomFlipAroundRoot
from mmpose.datasets.transforms.topdown_transforms import TopdownAffine
from torch.utils.data import DataLoader

cfg = Config.fromfile(
	r'C:\Users\user\Documents\GitHub\mmpose\my_code\custom_config\HMD_mo2cap2_imple_config.py'
)

In [2]:
cfg.train_dataloader
cfg.dataset_mo2cap2_train

{'type': 'Mo2Cap2CocoDataset',
 'data_root': 'F:\\mo2cap2_data_half\\TrainSet',
 'data_mode': 'topdown',
 'filter_cfg': {'filter_empty_gt': False, 'min_size': 32},
 'pipeline': [{'type': 'LoadImage'},
  {'type': 'GetBBoxCenterScale', 'padding': 1.0},
  {'type': 'TopdownAffine', 'input_size': (256, 256)},
  {'type': 'GenerateTarget',
   'encoder': {'type': 'Custom_mo2cap2_MSRAHeatmap',
    'input_size': (256, 256),
    'heatmap_size': (47, 47),
    'sigma': 3}},
  {'type': 'PackPoseInputs'}],
 'input_size': (256, 256)}

In [5]:

codec = dict(
	type='Custom_mo2cap2_MSRAHeatmap', input_size=(384, 384), heatmap_size=(47, 47), sigma=2)

# data
dataset_type = 'Mo2Cap2CocoDataset'
data_mode = 'topdown'
# data_root = r'C:\Users\user\Documents\GitHub\mmpose\data'

# ---
# # mo2cap2 dataset temp
# ann_file_val = r'F:\mo2cap2_data_temp_extracted\TestSet'
# ann_file_train = r'F:\mo2cap2_data_temp_extracted\TrainSet'
# ---
# # ## mo2cap2 dataset small, test small
# ann_file_test = r'F:\mo2cap2_data_small\TestSet'
# ann_file_val = r'F:\mo2cap2_data_small\ValSet'
# ann_file_train = r'F:\mo2cap2_data_small\TrainSet'
# ---
# mo2cap2 dataset train middel, test all
ann_file_test = r'F:\mo2cap2_data_half\TestSet'
ann_file_val = r'F:\mo2cap2_data_half\ValSet'
ann_file_train = r'F:\mo2cap2_data_half\TrainSet'
# ---
# # mo2cap2 dataset train all, test all
# ann_file_val = r'F:\extracted_mo2cap2_dataset\TestSet'
# ann_file_train = r'F:\extracted_mo2cap2_dataset\TrainSet'

####
# ann_file_test = r'F:\mo2cap2_one_data\TestSet'
# ann_file_val = r'F:\mo2cap2_one_data\ValSet'
# ann_file_train = r'F:\mo2cap2_one_data\TrainSet'
###

# ann_file_test = r'D:\cross_plat\mo2cap2_data_half\TestSet'
# ann_file_val = r'D:\cross_plat\mo2cap2_data_half\ValSet'
# ann_file_train = r'D:\cross_plat\mo2cap2_data_half\TrainSet'


train_pipeline = [
	LoadImage(),
	GetBBoxCenterScale(padding=1.),
	# dict(type='RandomFlip', direction='horizontal'), # 3d keypoints도 플립시켜야됨.
	# # dict(type='RandomHalfBody'),
	# # dict(type='RandomBBoxTransform'),
	TopdownAffine(input_size=codec['input_size']),
	GenerateTarget(encoder=codec),
	PackPoseInputs(),
]

val_pipeline = [
	LoadImage(),
	GetBBoxCenterScale(padding=1.),
	TopdownAffine(input_size=codec['input_size']),
	GenerateTarget(encoder=codec),
	PackPoseInputs(),
]

dataset_mo2cap2_train = dict(
	# type=dataset_type,
	data_root=ann_file_train,
	data_mode=data_mode,
	filter_cfg=dict(filter_empty_gt=False, min_size=32),
	pipeline=train_pipeline,
)

dataset_mo2cap2_val = dict(
	# type=dataset_type,
	data_root=ann_file_test,
	data_mode=data_mode,
	filter_cfg=dict(filter_empty_gt=False, min_size=32),
	# ann_file=ann_file_val,
	# data_prefix=dict(img=r'C:\Users\user\Documents\GitHub\mmpose\data\coco\train2017'),
	pipeline=val_pipeline,
	test_mode = True,
	input_size = codec['input_size']
)

In [6]:
# train_dataset = Mo2Cap2CocoDataset(**dataset_mo2cap2_train)
val_dataset = Mo2Cap2CocoDataset(**dataset_mo2cap2_val)

In [9]:
val_dataset[0]['inputs'].shape

torch.Size([3, 384, 384])

In [11]:
train_dataset[31999]['data_samples']

<PoseDataSample(

    META INFORMATION
    dataset_name: 'mo2cap2'
    img_shape: (256, 256)
    ori_shape: (256, 256)
    img_id: 32999
    input_center: array([128., 128.])
    flip_indices: [0, 4, 5, 6, 1, 2, 3, 11, 12, 13, 14, 7, 8, 9, 10]
    input_size: (256, 256)
    img_path: 'F:\\mo2cap2_data_half\\TrainSet\\mo2cap2_chunk_0032\\rgba\\mo2cap2_chunk_0032_000999.png'
    input_scale: array([256., 256.])
    raw_ann_info: 
        img_path: 'F:\\mo2cap2_data_half\\TrainSet\\mo2cap2_chunk_0032\\rgba\\mo2cap2_chunk_0032_000999.png'
        json_path: 'F:\\mo2cap2_data_half\\TrainSet\\mo2cap2_chunk_0032\\json\\mo2cap2_chunk_0032_000999.json'
        bbox: array([[  0,   0, 256, 256]])
        keypoints: array([[[131.56540762, 195.38776544],
                    [168.12443172, 173.36135359],
                    [183.40560259, 129.60919845],
                    [205.51045079,  94.66215272],
                    [ 87.7879861 , 185.06874965],
                    [ 72.66938765, 139.74631743

In [6]:
# temp_list = []
# for i in range(31999):	
# 	img_id = train_dataset[i]['data_samples'].img_id
# 	if img_id in temp_list:
# 		print(img_id)
# 		print(temp_list)
# 	else:
# 		temp_list.append(img_id)